<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/Projects/102-Class%C2%A0Flower%C2%A0Classification%20with%20Transfer%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dealing with 100+ classes, especially when some are visually similar (like different flower species), requires a more powerful technique than a simple CNN built from scratch. We'll use the Oxford Flowers 102 dataset and implement Transfer Learning using a pre-trained model like MobileNetV2.**



In [19]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [20]:
#  Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 102 # The Oxford Flowers 102 dataset has 102 classes
AUTOTUNE = tf.data.AUTOTUNE

In [22]:
# Load the Dataset
# Oxford Flowers 102 has 'train', 'validation', and 'test' splits
(train_ds_raw, val_ds_raw, test_ds_raw), metadata = tfds.load(
    'oxford_flowers102',
    split=['train', 'validation', 'test'],
    as_supervised=True,
    with_info=True,
)

print(f"Number of training examples: {metadata.splits['train'].num_examples}")
print(f"Number of classes: {metadata.features['label'].num_classes}")


Number of training examples: 1020
Number of classes: 102


In [23]:
#Preprocessing Function
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [24]:
# Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

In [25]:
# Create the Input Pipeline
def prepare_dataset(ds, shuffle=False, augment=False):
    ds = ds.map(preprocess, num_parallel_calls=AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(BATCH_SIZE)
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
    return ds.cache().prefetch(AUTOTUNE)


In [26]:
# Prepare the final datasets
train_ds = prepare_dataset(train_ds_raw, shuffle=True, augment=True)
val_ds = prepare_dataset(val_ds_raw)
test_ds = prepare_dataset(test_ds_raw)

In [27]:
# Build Model using Keras Applications MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,  # remove classifier head
    weights='imagenet'  # use pretrained weights
)
base_model.trainable = False

# Functional API
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs, name="mobilenetv2_flowers")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [28]:
# --- Compile Model ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)